In [31]:
import time
import networkx as nx
import pandas as pd
import numpy as np
with open('input.txt') as f:
    lines = f.read().splitlines()

## Part 1

In [37]:
starttime = time.perf_counter_ns()
# create matrix with all points
matrix = []
for line in lines:
    matrix.append(list(line))

pd.set_option('display.max_rows', 1000)
# create a dataframe with this matrix
df = pd.DataFrame(matrix)

for i in range(0, len(df)):
    if not df.iloc[i].str.contains('#').any():
        pass

# first let's expand the universe
# rows containing only '.' are duplicated
# columns containing only '.' are duplicated

# Duplicate rows
new_rows = []
for i in range(len(df)):
    new_rows.append(df.iloc[i])
    if not df.iloc[i].str.contains('#').any():
        # duplicate row
        new_rows.append(df.iloc[i])

# create new DataFrame
df = pd.DataFrame(new_rows).reset_index(drop=True)

# Duplicate columns
new_cols = pd.DataFrame()
for col in df.columns:
    new_cols = pd.concat([new_cols, df[col]], axis=1)
    if not df[col].str.contains('#').any():
        # duplicate column
        new_cols = pd.concat([new_cols, df[col]], axis=1)

# Reset column names
new_cols.columns = range(new_cols.shape[1])

# now we need to create a graph, where every point is a node, every node needs to be connected to all other nodes
# nodes with values '#' are added to a list of nodes
# Create a grid graph
universe = nx.grid_2d_graph(len(new_cols), len(new_cols.columns))

# Identify the galaxies
galaxies = [(i, j) for i in range(len(new_cols)) for j in range(len(new_cols.columns)) if new_cols.iloc[i, j] == '#']

# now we need to find the shortest path between all pairs of nodes
# we can use the Floyd-Warshall algorithm for this
total_weight = 0
visited = set()
for galaxy1 in galaxies:
    for galaxy2 in galaxies:
        if galaxy1 != galaxy2 and (galaxy1, galaxy2) not in visited:
            visited.add((galaxy1, galaxy2))
            visited.add((galaxy2, galaxy1))
            #path = nx.shortest_path(universe, galaxy1, galaxy2)
            # calcualte the length by subtracting the coordinates
            path = abs(galaxy1[0] - galaxy2[0]) + abs(galaxy1[1] - galaxy2[1])
            total_weight += path
print("Length total: ", total_weight)
print("Time taken: ", round((time.perf_counter_ns() - starttime)/1000000, 3), "ms")

Length total:  374
Time taken:  13.072 ms


## Part 2